# Primo + Pensieve

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

SAVEPATH = "./"

sns.set_style("ticks")
font = {
    "font.size": 12,
}
sns.set_style(font)
paper_rc = {
    "lines.linewidth": 3,
    "lines.markersize": 10,
}
sns.set_context("paper", font_scale=2, rc=paper_rc)
current_palette = sns.color_palette()


def autolabel(rects, ax, precision=3):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate(
            f"%.{precision}f" % height,
            xy=(rect.get_x() + rect.get_width() / 2, height),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha="center",
            va="bottom",
            size=14,
        )

In [ ]:
TRACE = 'HSDPA'
# TRACE = 'FCC'

if TRACE == 'HSDPA':
    RESULTS_FOLDER = "./baselines/results/"
elif TRACE == 'FCC':
    RESULTS_FOLDER = "./baselines/results_fcc/"
else:
    raise ValueError('Wrong Input!')

NUM_BINS = 1000
BITS_IN_BYTE = 8.0
MILLISEC_IN_SEC = 1000.0
M_IN_B = 1000000.0
VIDEO_LEN = 48
VIDEO_BIT_RATE = [300, 750, 1200, 1850, 2850, 4300]
# SCHEMES = ["sim_rl", "sim_metis", "sim_m200", "sim_primo"]
SCHEMES = ["Primo", "RL", "Metis", "MPC", "BB"]

cmp = sns.color_palette("tab10")
files = sorted(os.listdir(RESULTS_FOLDER))
log_files = []
lower_schemes = [s.lower() for s in SCHEMES]
for f in files:
    alg_name = f.split('_')[2]
    if alg_name in lower_schemes:
        log_files.append(f)

all_trace = pd.DataFrame()

for i in range(len(log_files)):
    f = log_files[i]
    df = pd.read_csv(RESULTS_FOLDER+f, sep='\t', names=['time_ms', 'bit_rate', 'temp1', 'buff', 'bw', 'temp2', 'reward'],header=None)
    df['bw'] = df['bw'] / df['temp2'] * BITS_IN_BYTE * MILLISEC_IN_SEC / M_IN_B
    df['time_ms'] = df['time_ms'] - df.iloc[0]['time_ms']

    for scheme in lower_schemes:
        if scheme in f.split('_')[2]:
            df['algorithm'] = scheme
            df['video_len'] = len(df)
            df.drop(index=0, inplace=True)  # Avoid start influence?

    idx = i % (len(log_files) // len(lower_schemes))
    df['trace_idx'] = idx

    all_trace = pd.concat([all_trace, df])
    # all_trace['video_len'].unique()

In [ ]:
summary_df = all_trace.groupby(['trace_idx', 'algorithm']).mean()['reward'].reset_index()
summary = pd.DataFrame()
for alg in lower_schemes:
    alg_reward = summary_df[summary_df['algorithm']==alg]['reward'].reset_index(drop=True)
    summary = pd.concat([summary, alg_reward.rename(alg)], axis=1)

summary = summary.reset_index(drop=True)

### HSDPA

In [ ]:
def pensieve_cdf_average(trace, save=False):
    grid_params=dict(width_ratios=[1.5, 1])
    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, gridspec_kw=grid_params, constrained_layout=True, figsize=(10, 4))

    linestyle_list = ['-', '--', ':', '-.', ':']
    labels = []
    SCHEMES[1] = 'Pensieve'

    for i in range(len(SCHEMES)):
        values, base = np.histogram(summary.iloc[:,i], bins=NUM_BINS)
        cumulative = np.cumsum(values) / len(summary) * 100
        ax1.plot(base[:-1], cumulative, linewidth=2.5, linestyle=linestyle_list[i], label=SCHEMES[i])
    ax1.set_xlabel(f"Average QoE")
    ax1.set_ylabel(f"Fraction (%)")
    ax1.set_ylim(-0.5, 100)

    if trace == 'HSDPA':
        ax1.set_xlim(-0.3, 2.5)
    else:
        ax1.set_xlim(-0.3, 3.5)

    
    ax1.grid(axis="y", linestyle=":")
    ax1.legend(fontsize=16)

    x = np.arange(1, 6)
    width = 0.75
    p = ax2.bar(x, summary.mean(), width=width, alpha=0.8, linewidth=1, edgecolor="k", color = cmp[0:5])

    ax2.set_ylabel(f"Average QoE")
    ax2.set_xticks(x)
    ax2.set_xticklabels(SCHEMES, rotation=30)
    ax2.set_ylim(0, 1.2)
    sns.despine()
    autolabel(p, ax2, precision=3)

    if save:
        fig.savefig(f"{SAVEPATH}/Pensieve_CDF_Average_{trace}.png", bbox_inches="tight")

# pensieve_cdf_average(trace='HSDPA', save=True)
pensieve_cdf_average(trace=TRACE, save=True)

### FCC

In [ ]:
# TRACE = 'HSDPA'
TRACE = 'FCC'

if TRACE == 'HSDPA':
    RESULTS_FOLDER = "./baselines/results/"
elif TRACE == 'FCC':
    RESULTS_FOLDER = "./baselines/results_fcc/"
else:
    raise ValueError('Wrong Input!')

NUM_BINS = 1000
BITS_IN_BYTE = 8.0
MILLISEC_IN_SEC = 1000.0
M_IN_B = 1000000.0
VIDEO_LEN = 48
VIDEO_BIT_RATE = [300, 750, 1200, 1850, 2850, 4300]
# SCHEMES = ["sim_rl", "sim_metis", "sim_m200", "sim_primo"]
SCHEMES = ["Primo", "RL", "Metis", "MPC", "BB"]

cmp = sns.color_palette("tab10")
files = sorted(os.listdir(RESULTS_FOLDER))
log_files = []
lower_schemes = [s.lower() for s in SCHEMES]
for f in files:
    alg_name = f.split('_')[2]
    if alg_name in lower_schemes:
        log_files.append(f)

all_trace = pd.DataFrame()

for i in range(len(log_files)):
    f = log_files[i]
    df = pd.read_csv(RESULTS_FOLDER+f, sep='\t', names=['time_ms', 'bit_rate', 'temp1', 'buff', 'bw', 'temp2', 'reward'],header=None)
    df['bw'] = df['bw'] / df['temp2'] * BITS_IN_BYTE * MILLISEC_IN_SEC / M_IN_B
    df['time_ms'] = df['time_ms'] - df.iloc[0]['time_ms']

    for scheme in lower_schemes:
        if scheme in f.split('_')[2]:
            df['algorithm'] = scheme
            df['video_len'] = len(df)
            df.drop(index=0, inplace=True)  # Avoid start influence?

    idx = i % (len(log_files) // len(lower_schemes))
    df['trace_idx'] = idx

    all_trace = pd.concat([all_trace, df])
    # all_trace['video_len'].unique()

In [ ]:
summary_df = all_trace.groupby(['trace_idx', 'algorithm']).mean()['reward'].reset_index()
summary = pd.DataFrame()
for alg in lower_schemes:
    alg_reward = summary_df[summary_df['algorithm']==alg]['reward'].reset_index(drop=True)
    summary = pd.concat([summary, alg_reward.rename(alg)], axis=1)

summary = summary.reset_index(drop=True)

In [ ]:
def pensieve_cdf_average(trace, save=False):
    grid_params=dict(width_ratios=[1.5, 1])
    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, gridspec_kw=grid_params, constrained_layout=True, figsize=(10, 4))

    linestyle_list = ['-', '--', ':', '-.', ':']
    labels = []
    SCHEMES[1] = 'Pensieve'

    for i in range(len(SCHEMES)):
        values, base = np.histogram(summary.iloc[:,i], bins=NUM_BINS)
        cumulative = np.cumsum(values) / len(summary) * 100
        ax1.plot(base[:-1], cumulative, linewidth=2.5, linestyle=linestyle_list[i], label=SCHEMES[i])
    ax1.set_xlabel(f"Average QoE")
    ax1.set_ylabel(f"Fraction (%)")
    ax1.set_ylim(-0.5, 100)

    if trace == 'HSDPA':
        ax1.set_xlim(-0.3, 2.5)
    else:
        ax1.set_xlim(-0.3, 3.5)

    
    ax1.grid(axis="y", linestyle=":")
    ax1.legend(fontsize=16)

    x = np.arange(1, 6)
    width = 0.75
    p = ax2.bar(x, summary.mean(), width=width, alpha=0.8, linewidth=1, edgecolor="k", color = cmp[0:5])

    ax2.set_ylabel(f"Average QoE")
    ax2.set_xticks(x)
    ax2.set_xticklabels(SCHEMES, rotation=30)
    ax2.set_ylim(0, 1.2)
    sns.despine()
    autolabel(p, ax2, precision=3)

    if save:
        fig.savefig(f"{SAVEPATH}/Pensieve_CDF_Average_{trace}.png", bbox_inches="tight")

# pensieve_cdf_average(trace='HSDPA', save=True)
pensieve_cdf_average(trace=TRACE, save=True)